## E-76 Obligated Federal Funds Data Analysis

E-76 Obligated Federal Funds Data Exploration
From the Division of Local Assistance
E-76's or "Authorizations to Proceed" is a federal authorization of funds to establish the reimbursement date for a phase of work

#### Question
- What types of projects are being funded?
- Is there a type of funding that is more likely to be approved?
- What is the most/least common type of funding

#### Data Sources:¶
* [Waiting dataset](https://dot.ca.gov/programs/local-assistance/reports/e-76-waiting)
* [Obligated dataset](https://dot.ca.gov/programs/local-assistance/reports/e-76-obligated)

In this notebook, we will conduct analysis to determine what types of projects are funded through DLA

In [85]:
## one time download
#! pip install plotnine

In [86]:
import pandas as pd
from siuba import _, mutate, count, filter, group_by, ungroup, summarize
import numpy as np

In [87]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:.2f}".format

In [88]:
df = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/clean_waiting_obligated.csv', low_memory=False)


In [89]:
df.head()

,Unnamed: 0,location,prefix,project_no,agency,prepared_date,submit_to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,ProjectID,ProjectNO,projectID,projectNO,compare_id_locode
0,0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NONMPO,NaN,5904,121,5904,121,True
1,1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NONMPO,NaN,32D0,8,32D0,8,False
2,2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NONMPO,NaN,4820,4,4820,4,False
3,3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,5924,244,5924,244,True
4,4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,5924,214,5924,214,True


In [90]:
df = df.drop('ProjectID', axis=1)
df = df.drop('ProjectNO', axis=1)


In [91]:
df.mpo.unique()

array(['NONMPO', 'SACOG', 'MTC', 'TCAG', 'KCOG', 'CFCG', 'SCAG', 'BCAG',
       'SLAC', 'STANCOG', 'SJCG', 'SDAG', 'KCAG', 'SBCG', 'MCAG', 'TMPO',
       'AMBAG', 'MCTC', 'STPA', 'COFCG', 'FCOG', 'ER NONE', 'SHASTA',
       'MNOLTC', 'SJCOG', 'STACOG', 'STNCOG', 'FSTIP', 'SANDAG', 'SANDA',
       'SLOCOG', nan], dtype=object)

In [92]:
df.sample(5)

,Unnamed: 0,location,prefix,project_no,agency,prepared_date,submit_to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
14278,14278,Obligated,BPMPL,5957(101),San Diego County,2019-02-20,2019-02-20,2019-02-20,2019-02-20,2019-02-25,0.00,0.00,0.00,Authorized,5957,11,E-76 approved on,15.00,0.00,5.00,CAL44,Various Off Fed-aid System Bridges,Bridge Preventive Maintenance,4,2019-02-05,2019-02-06,SDAG,NaN,5957,101,True
15726,15728,Obligated,CML,5933(109),Alameda County,2020-01-27,2020-01-27,2020-01-27,2020-01-27,2020-01-28,-20611.05,0.00,-315458.10,Authorized,5933,4,E-76 approved on,4.00,0.00,1.00,REG110013,Various Locations,Purchase Of Electric Vehicles,4,2020-01-23,2020-01-24,MTC,NaN,5933,109,True
13059,13059,Obligated,ER,32L0(300),Humboldt County,2018-12-07,2018-12-07,2019-01-11,2019-01-14,2019-01-16,6020.04,0.00,6800.00,Authorized,5904,1,E-76 approved on,1.00,38.00,2.00,NaN,Alderpoint Road Pm. 23.75,Repair Storm Water Debris Damage To Culvert An...,1,2018-12-06,2018-12-06,NONMPO,NaN,32L0,300,False
2868,2868,Obligated,ACSTP,32L0(347),Napa County,2017-10-05,2017-11-01,2017-12-11,2017-12-20,2017-12-22,0.00,300149.00,338000.00,Authorized,5921,4,E-76 approved on,0.00,49.00,2.00,NaN,Silverado Trail Mpm 18.02,Eo Agency - Temporary Wall Constructed. See At...,1,2017-10-03,2017-11-01,MTC,NaN,32L0,347,False
1966,1966,Obligated,ER,20A0(008),Humboldt County,2018-03-30,2018-03-30,2018-04-05,2018-04-05,2018-04-13,8945.35,0.00,10103.75,Authorized,5904,1,E-76 approved on,2.00,6.00,8.00,NaN,"Alderpoint Road, Pm 14.10",Storm Damage Restoration,4,2018-03-28,2018-03-28,NONMPO,NaN,20A0,8,False


In [93]:
df.prefix.nunique()

392

In [94]:
df.prefix.unique()

array(['BPMP', 'ER', 'CML', 'BRLO', 'BRLS', 'HSIPLN', 'STPL', 'HSIPL',
       'BPMPL', 'HPLULN', 'RPSTPL', 'HRRRL', 'ACSTP', 'ATPCML', 'BRLSZD',
       'ATPL', 'STPLZ', 'FERPL16', 'BHLS', 'BHLO', 'ATPLNI', 'HP21L',
       'STPTCIF', 'STPCML', 'STPLNI', 'VPPL', 'NCIIPL', 'DEM06U', 'HSIP',
       'STPLX', 'IMDLN', 'BHLSZD', 'SRTSL', 'RSTPL', 'CMSTPL', 'PNRSTCI',
       'CMLNI', 'BHLSZ', 'FSPCML', 'ATPSB1L', 'ATCMTD', 'DEM10L',
       'HSIIPL', 'HPLUL', 'ATP-CML', 'BRLOZE', 'CASB902', 'INVEST',
       'SRTS', 'SRTSLNI', 'ATPHSIP', 'NMTPL', 'HSIPCML', 'STPLN',
       'STPLUL', 'PNRSLN', 'CMLN', 'BR', 'BRLSZ', 'PNRS', 'RPHP21L',
       'STPLR', 'LPPSB1L', 'TCSP', 'FERSTP', 'PLHDL06', 'ATPSB1', 'LTAP',
       'FTAATPL', 'FTACML', 'BRLOZ', 'BRLKS', 'PRNSL', 'BRNBIF', 'DEMO6L',
       'STP', 'DEM08L', 'PNRSL', 'HSIPSTP', 'FTAFER16', 'CASB06',
       'BRLOZB', 'CMLFSP', 'NHCBPL', 'STPLER', 'STPLHS', 'HPPLU',
       'DEM10A', 'CMTCIFL', 'CRP', 'DEMO10', 'HPLU', 'DEM06L', 'DEMO17',
       'DBF', 

In [95]:
df.prefix.value_counts()

CML        3334
HSIPL      3331
STPL       2810
ER         1607
BRLO       1416
           ... 
CASB09        1
DEMO11        1
TCIFESP       1
RPSTPL-       1
FTACRRS       1
Name: prefix, Length: 392, dtype: int64

In [96]:
df.prefix.replace(['BPMP'], ['BPMPL'], inplace=True)

In [97]:
df.prefix.replace(['ATPL'], ['ATP'], inplace=True)

In [98]:
df.prefix.replace(['VPPPL'], ['VPPP'], inplace=True)

In [99]:
df.prefix.replace(['ATP-CML'], ['ATPCML'], inplace=True)

In [100]:
df.prefix.replace(['ATPL'], ['ATP'], inplace=True)

In [101]:
df.prefix.replace(['CASB003'], ['CASB'], inplace=True)
df.prefix.replace(['CASB05'], ['CASB'], inplace=True)
df.prefix.replace(['CASB06'], ['CASB'], inplace=True)
df.prefix.replace(['CASB07'], ['CASB'], inplace=True)
df.prefix.replace(['CASB09'], ['CASB'], inplace=True)
df.prefix.replace(['CASB10'], ['CASB'], inplace=True)
df.prefix.replace(['CASB11'], ['CASB'], inplace=True)
df.prefix.replace(['CASB12'], ['CASB'], inplace=True)
df.prefix.replace(['CASB803'], ['CASB'], inplace=True)
df.prefix.replace(['CASB902'], ['CASB'], inplace=True)
df.prefix.replace(['CASB904'], ['CASB'], inplace=True)
df.prefix.replace(['CASB905'], ['CASB'], inplace=True)
df.prefix.replace(['CASB12'], ['CASB'], inplace=True)
df.prefix.replace(['CASBL'], ['CASB'], inplace=True)

In [102]:



df.prefix.replace(['BR-'], ['BR'], inplace=True)

In [103]:
df.prefix.replace(['DEM03L'], ['DEM'], inplace=True)

In [104]:
df.prefix.replace(['FERPL16'], ['FERPL'], inplace=True)
df.prefix.replace(['FERPL17'], ['FERPL'], inplace=True)
df.prefix.replace(['FERPL18'], ['FERPL'], inplace=True)
df.prefix.replace(['FERPL19'], ['FERPL'], inplace=True)
df.prefix.replace(['FERPLN16'], ['FERPL'], inplace=True)

In [105]:
df.prefix.replace(['FSP11'], ['FSP'], inplace=True)
df.prefix.replace(['FSP12'], ['FSP'], inplace=True)
df.prefix.replace(['FSP13'], ['FSP'], inplace=True)
df.prefix.replace(['FSP14'], ['FSP'], inplace=True)

In [106]:
df.prefix.replace(['HRRRL'], ['HRRR'], inplace=True)
df.prefix.replace(['IMDL'], ['IMD'], inplace=True)
df.prefix.replace(['HSIPSTPL'], ['HSIPSTP'], inplace=True)
df.prefix.replace(['PNRSTCIL'], ['PNRSTCI'], inplace=True)
df.prefix.replace(['RSTPL'], ['RSTP'], inplace=True)
df.prefix.replace(['RPSTPLE-'], ['RPSTPLE'], inplace=True)
df.prefix.replace(['TILUL'], ['TILU'], inplace=True)
df.prefix.replace(['STPL'], ['STP'], inplace=True)
df.prefix.replace(['RPSTPLE-'], ['RPSTPLE'], inplace=True)

In [107]:
df.prefix.replace(['HSIPL'], ['HSIP'], inplace=True)
df.prefix.replace(['HSIP5'], ['HSIP'], inplace=True)
df.prefix.replace(['HSIIPL'], ['HSIP'], inplace=True)


df.prefix.replace(['NCPDL0'], ['NCPD'], inplace=True)
df.prefix.replace(['NCPD02'], ['NCPD'], inplace=True)

df.prefix.replace(['FERPLN16'], ['FERPL'], inplace=True)

In [108]:
df.prefix.replace(['HISP5'], ['HSIP'], inplace=True)

In [109]:
df.prefix.replace(['PLHL04'], ['PLHL'], inplace=True)
df.prefix.replace(['PLHL05'], ['PLHL'], inplace=True)
df.prefix.replace(['PLHL10'], ['PLHL'], inplace=True)
df.prefix.replace(['PLHL11'], ['PLHL'], inplace=True)

df.prefix.replace(['PLHDL05'], ['PLDHL'], inplace=True)
df.prefix.replace(['PLHDL06'], ['PLDHL'], inplace=True)
df.prefix.replace(['PLHDL08'], ['PLDHL'], inplace=True)

In [110]:
df.prefix.replace(['PLHL04'], ['PLHL'], inplace=True)
df.prefix.replace(['PLHL05'], ['PLHL'], inplace=True)

In [111]:
df.prefix.replace(['RPSTPL'], ['RPSTP'], inplace=True)
df.prefix.replace(['RPSTPL-'], ['RPSTP'], inplace=True)

In [112]:
df.prefix.replace(['SRTSL'], ['SRTS'], inplace=True)
df.prefix.replace(['SRTSL-'], ['SRTS'], inplace=True)

In [113]:
df.prefix.replace(['TGR2DGL'], ['TGR2DG'], inplace=True)
df.prefix.replace(['TGR2DG.'], ['TGR2DG'], inplace=True)

In [114]:
df.prefix.replace(['TCSPL'], ['TCSP'], inplace=True)
df.prefix.replace(['TCSP02'], ['TCSP'], inplace=True)
df.prefix.replace(['TCSP03'], ['TCSP'], inplace=True)
df.prefix.replace(['TCSP05'], ['TCSP'], inplace=True)
df.prefix.replace(['TCSP06'], ['TCSP'], inplace=True)
df.prefix.replace(['TCSP08'], ['TCSP'], inplace=True)
df.prefix.replace(['TCSP10'], ['TCSP'], inplace=True)

In [115]:
df.prefix.nunique()

335

In [116]:
print(sorted(df['prefix'].unique()))

['6140000', 'ACNHPI', 'ACST-ER', 'ACSTP', 'ASCTP', 'ATCMTD', 'ATF5512', 'ATP', 'ATPCML', 'ATPHIPL', 'ATPHSIP', 'ATPLN', 'ATPLNI', 'ATPLSB1', 'ATPNIL', 'ATPSB1', 'ATPSB1L', 'ATPSTC', 'ATPSTCML', 'ATPSTPL', 'BDGL', 'BGFL', 'BHL', 'BHLO', 'BHLOZ', 'BHLOZD', 'BHLS', 'BHLSCR', 'BHLSP', 'BHLSZ', 'BHLSZD', 'BPMPL', 'BR', 'BR-NBISL', 'BRCMLO', 'BRHIPL', 'BRIM', 'BRL', 'BRLKS', 'BRLNS', 'BRLO', 'BRLOZ', 'BRLOZA', 'BRLOZB', 'BRLOZE', 'BRLRT', 'BRLS', 'BRLSN', 'BRLSZ', 'BRLSZA', 'BRLSZD', 'BRLZ', 'BRNBIF', 'BRNBISL', 'BROS', 'BUILDL', 'CASB', 'CFLBRLO', 'CMAQL', 'CMESPL', 'CMFERP', 'CMFERP16', 'CMHIPL', 'CMHPLU', 'CMHPLULN', 'CMHRL', 'CMHSIPL', 'CML', 'CMLFSP', 'CMLFSP18', 'CMLFTA', 'CMLG', 'CMLGNI', 'CMLHPL', 'CMLN', 'CMLNGI', 'CMLNI', 'CMLSBCG', 'CMLSTBGL', 'CMRPSTPL', 'CMSTPL', 'CMSTPLN', 'CMSTPLNI', 'CMTCIFL', 'CMTCIFLN', 'CMTLSP', 'CRDL', 'CRP', 'CRRSAA', 'DBF', 'DBPL02', 'DE', 'DEM', 'DEM04L', 'DEM05', 'DEM05L', 'DEM05LN', 'DEM05U', 'DEM05UBL', 'DEM06', 'DEM06L', 'DEM06U', 'DEM06UBL', 'DEM0

In [117]:
df.prefix.value_counts()

HSIP       3367
CML        3334
STP        2817
ER         1607
BRLO       1416
           ... 
BROS          1
ITS13         1
FBDLNI        1
TCIFESP       1
FTACRRS       1
Name: prefix, Length: 335, dtype: int64

In [118]:
(df
  >> group_by(_.prefix)
  >> summarize(averge_funds = _.total_requested.mean())
  )


,prefix,averge_funds
0,6140000,645621.50
1,ACNHPI,14713175.00
2,ACST-ER,491842.88
3,ACSTP,335628.39
4,ASCTP,15000.00
...,...,...
330,USFSSTPL,264444.50
331,VPPL,162853.20
332,VPPP,144039.36
333,VPPTCS,-443858.28
